In [1]:
import torch

In [2]:
! conda env list
! source activate <name>
! source deactivate <name>

# conda environments:
#
base                     /home/ec2-user/anaconda3
JupyterSystemEnv         /home/ec2-user/anaconda3/envs/JupyterSystemEnv
R                        /home/ec2-user/anaconda3/envs/R
amazonei_pytorch_latest_p37     /home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37
mxnet_p38                /home/ec2-user/anaconda3/envs/mxnet_p38
python3                  /home/ec2-user/anaconda3/envs/python3
pytorch_p39           *  /home/ec2-user/anaconda3/envs/pytorch_p39
tensorflow2_p310         /home/ec2-user/anaconda3/envs/tensorflow2_p310

/bin/sh: -c: line 0: syntax error near unexpected token `newline'
/bin/sh: -c: line 0: ` source activate <name>'
/bin/sh: -c: line 0: syntax error near unexpected token `newline'
/bin/sh: -c: line 0: ` source deactivate <name>'


In [3]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0


In [4]:
import boto3
region = boto3.Session().region_name
image_uri = "763104351884.dkr.ecr.{}.amazonaws.com/pytorch-training".format(region)
image_uri

'763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-training'

In [5]:
print (torch.cuda.is_available())
print (torch.cuda.device_count())
print (torch.cuda.current_device())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device.type)

True
1
0
cuda


In [6]:
import os

In [19]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0, 1"

In [21]:
%%time
import torch
import torch.nn as nn
dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
#dev = "cpu"
t1 = torch.randn(1000,1000)
t2 = torch.randn(1000,1000).to(dev)
print(t1)  # tensor([[-0.2678,  1.9252]])
print(t2)  # tensor([[ 0.5117, -3.6247]], device='cuda:0')
t1.to(dev)
print(t1)  # tensor([[-0.2678,  1.9252]])
print(t1.is_cuda) # False
t1 = t1.to(dev)
print(t1)  # tensor([[-0.2678,  1.9252]], device='cuda:0')
print(t1.is_cuda) # True

class M(nn.Module):
    def __init__(self):        
        super().__init__()        
        self.l1 = nn.Linear(1000,2)

    def forward(self, x):                      
        x = self.l1(x)
        return x
model = M()   # not on cuda
model = model.to(dev) # is on cuda (all parameters)
print(next(model.parameters()).is_cuda) # True

for i in range(100000):
    res = model.forward(t2)
    #if i %100 ==0: print (res.is_cuda)

tensor([[-0.3544,  2.8961, -0.4621,  ..., -1.9107,  1.4183,  0.2584],
        [ 1.2220,  1.1107, -0.4341,  ...,  0.7482, -0.4176, -0.9539],
        [ 0.9947, -0.3699, -1.6086,  ...,  0.1043,  0.8148, -0.1422],
        ...,
        [-0.2547,  0.6305, -1.2029,  ..., -0.5115, -2.7221, -0.0522],
        [-0.2941, -0.1278,  0.2114,  ..., -0.5410, -2.1334, -0.2985],
        [ 1.8878,  0.4411,  0.7302,  ...,  0.8559,  2.6896, -0.9066]])
tensor([[-0.2035,  0.4134,  0.4755,  ..., -0.6031, -0.4613,  1.2219],
        [-1.6342,  0.2688, -0.5347,  ...,  0.1952, -0.3630, -0.8671],
        [-0.5760,  0.9896,  0.4382,  ...,  0.5892,  0.2498, -1.6362],
        ...,
        [ 0.6148,  0.0661, -1.6801,  ..., -0.4993, -1.1065, -1.0850],
        [ 1.1199,  0.1531, -1.8302,  ..., -0.5935, -1.1736,  0.4272],
        [-0.1980,  0.3656,  0.2120,  ...,  2.0770,  0.7935, -0.3178]],
       device='cuda:0')
tensor([[-0.3544,  2.8961, -0.4621,  ..., -1.9107,  1.4183,  0.2584],
        [ 1.2220,  1.1107, -0.4341,  .

In [45]:
res = ''
model = ''
ti=''
torch.cuda.empty_cache()

In [46]:
import gc
gc.collect()

0